In [1]:
import onnx

In [4]:
onnx_model = onnx.load('dsads_attn_conv_4.onnx')

In [5]:
def store_indices_and_names(onnx_model):
    indices_and_names = [(i, node.name) for i, node in enumerate(onnx_model.graph.node)]
    return indices_and_names

In [6]:
def get_by_name(container, name, name_field="name"):
    """Return item from protobuf container by .name field if it exists, None otherwise.
    Will throw an Exception if multiple items are found, since this violates the
    ONNX standard."""
    names = [getattr(x, name_field) for x in container]

    inds = [i for i, e in enumerate(names) if e == name]
    if len(inds) > 1:
        print(name)
        raise Exception("Found multiple get_by_name matches, undefined behavior")
    elif len(inds) == 0:
        return None
    else:
        ind = inds[0]
        return container[ind]

In [7]:
def get_by_name_multiple(container, name, name_field="name"):

    names = [getattr(x, name_field) for x in container]

    inds = [i for i, e in enumerate(names) if e == name]
    if len(inds) > 1:
        ind = inds[0]
        return container[ind]
    elif len(inds) == 0:
        return None
    else:
        ind = inds[0]
        return container[ind]

In [8]:
def rename_tensor(model, old_name, new_name):
        """Renames a tensor from old_name to new_name."""
        graph = model.graph
        # sweep over inputs
        if get_by_name(graph.input, old_name) is not None:
            get_by_name(graph.input, old_name).name = new_name
        # sweep over outputs
        if get_by_name(graph.output, old_name) is not None:
            get_by_name(graph.output, old_name).name = new_name
        # sweep over value_info
        if get_by_name(graph.value_info, old_name) is not None:
            get_by_name(graph.value_info, old_name).name = new_name
        # sweep over initializers
        if get_by_name(graph.initializer, old_name) is not None:
            get_by_name(graph.initializer, old_name).name = new_name
        # sweep over quantization annotations
        
        if get_by_name_multiple(graph.quantization_annotation, old_name, "tensor_name") is not None:
            get_by_name_multiple(graph.quantization_annotation, old_name, "tensor_name").tensor_name = new_name
        # sweep over node i/o
        for n in graph.node:
            if old_name in n.input:
                n.input[list(n.input).index(old_name)] = new_name
            if old_name in n.output:
                n.output[list(n.output).index(old_name)] = new_name

In [9]:
def GiveUniqueNodeNames(onnx_model):
        optype_count = {}
        for n in onnx_model.graph.node:
            if n.op_type not in optype_count.keys():
                optype_count[n.op_type] = 0
            n.name = "%s_%d" % (n.op_type, optype_count[n.op_type])
            optype_count[n.op_type] += 1
        # return model_was_changed = False as single iteration is always enough
        return onnx_model

In [10]:
def get_all_tensor_names(model):
        """Returns a list of all (input, output and value_info) tensor names
        in the graph."""
        graph = model.graph
        names = [x.name for x in graph.value_info]
        names += [x.name for x in graph.input]
        names += [x.name for x in graph.output]
        return names

In [11]:
import string
import random

def random_string(stringLength=6):
    """Randomly generate a string of letters and digits."""
    lettersAndDigits = string.ascii_letters + string.digits
    return "".join(random.choice(lettersAndDigits) for i in range(stringLength))

In [12]:
def GiveRandomTensorNames(model):
    names = get_all_tensor_names(model)
    for name in names:
        rename_tensor(model,name, random_string())
    return model

In [13]:
import onnx.numpy_helper as np_helper

def get_initializer(model, tensor_name, return_dtype=False):
        """Gets the initializer value for tensor with given name, if any.
        ret_dtype can be set to True to retrieve the TensorProto.DataType of the
        initializer by returning it as a second element of a tuple."""
        graph = model.graph
        init_names = [x.name for x in graph.initializer]
        try:
            init_ind = init_names.index(tensor_name)
            ret = np_helper.to_array(graph.initializer[init_ind])
            ret_dtype = graph.initializer[init_ind].data_type
            if return_dtype:
                return (ret, ret_dtype)
            else:
                return ret
        except ValueError:
            if return_dtype:
                return (None, None)
            else:
                return None


In [14]:
def GiveReadableTensorNames(model):
    model = GiveRandomTensorNames(model)
    graph = model.graph
    for n in graph.node:
            assert n.name != "", "Found empty node name"
            out_num = 0
            for o in n.output:
                rename_tensor(model,o, "%s_out%d" % (n.name, out_num))
                out_num += 1
            init_in_num = 0
            for i in n.input:
                if get_initializer(model, i) is not None:
                    rename_tensor(model, i, "%s_param%d" % (n.name, init_in_num))
                    init_in_num += 1
    # give special names to the model inputs and outputs
    for i, inp in enumerate(model.graph.input):
        iname = "global_in" if i == 0 else "global_in_%d" % i
        rename_tensor(model, inp.name, iname)
    for i, outp in enumerate(model.graph.output):
        oname = "global_out" if i == 0 else "global_out_%d" % i
        rename_tensor(model, outp.name, oname)
        # return model_was_changed = False as single iteration is always enough
    return model

def constant_output_handler(onnx_model):
    for i in range(len(onnx_model.graph.node)):
        init_in_num = 0
        for j in range(len(onnx_model.graph.node[i].input)):
            
            if get_initializer(model, i) is not None:
                
                if "Constant" in onnx_model.graph.node[i].input[j]:
                    
                init_in_num += 1
            
            if "Constant" in onnx_model.graph.node[i].input[j]:
                #print('-'*60)
                input_name = onnx_model.graph.node[i].input[j]
                #print(input_name)
                new_name = f"{onnx_model.graph.node[i].name}_param0" 
                """
                for k in range(len(onnx_model.graph.initializer)):
                    init_name = onnx_model.graph.initializer[k].name
                    if input_name == init_name:
                        #print(init_name)
                        onnx_model.graph.initializer[i].name = f"{onnx_model.graph.node[i].name}_param0"
                """
                for l in range(len(onnx_model.graph.quantization_annotation)):
                    tensor_name = onnx_model.graph.initializer[k].tensor_name
                    if input_name == tensor_name:
                        #print(init_name)
                        tensor_name = onnx_model.graph.initializer[k].tensor_name = f"{onnx_model.graph.node[i].name}_param0"
                
                onnx_model.graph.node[i].input[j] = f"{onnx_model.graph.node[i].name}_param0" 

def constant_output_handler(onnx_model):
    for i in range(len(onnx_model.graph.node)):
        init_in_num = 0
        for j in range(len(onnx_model.graph.node[i].input)):
            if "Constant" in onnx_model.graph.node[i].input[j]:
                #print('-'*60)
                input_name = onnx_model.graph.node[i].input[j]
                #print(input_name)
                new_name = f"{onnx_model.graph.node[i].name}_param0" 
                """
                for k in range(len(onnx_model.graph.initializer)):
                    init_name = onnx_model.graph.initializer[k].name
                    if input_name == init_name:
                        #print(init_name)
                        onnx_model.graph.initializer[i].name = f"{onnx_model.graph.node[i].name}_param0"
                """
                for l in range(len(onnx_model.graph.quantization_annotation)):
                    tensor_name = onnx_model.graph.initializer[k].tensor_name
                    if input_name == tensor_name:
                        #print(init_name)
                        tensor_name = onnx_model.graph.initializer[k].tensor_name = f"{onnx_model.graph.node[i].name}_param0"
                
                onnx_model.graph.node[i].input[j] = f"{onnx_model.graph.node[i].name}_param0" 


In [15]:
"""
for i in onnx_model.graph.quantization_annotation:
    print('-'*60)
    print(i.tensor_name)
    name = random_string()
    i.tensor_name = name
    print(name)
"""
onnx_model = GiveUniqueNodeNames(onnx_model)
onnx_model = GiveReadableTensorNames(onnx_model)
onnx_model = onnx.save(onnx_model, 'dsads_attn_conv_4_mod.onnx')

In [16]:
for i in range(len(onnx_model.graph.quantization_annotation)):
    name = "/channel_interaction/query/export_handler/Constant_output_0"
    if(onnx_model.graph.quantization_annotation[i].tensor_name == name):
        print(onnx_model.graph.quantization_annotation[i])

AttributeError: 'NoneType' object has no attribute 'graph'

In [ ]:
def find_i_by_prefix(onnx_model, name):
    indices_and_names = [(i, node.name) for i, node in enumerate(onnx_model.graph.node)]
    found_i_list = []

    # Extract the node_name from the given name (assuming it follows the pattern {node_name}_output_{anything})
    node_name = name.split("_output_")[0]

    for i, node_name_from_list in indices_and_names:
        if node_name_from_list == node_name:
            found_i_list.append(i)

    if len(found_i_list) == 1:
        return found_i_list[0]
    elif len(found_i_list) == 0:
        raise Exception("No matching elements found.")
    else:
        raise Exception("Multiple matching elements found.")

In [ ]:
def rename_nodes_by_type(onnx_model, node_type):
    assert node_type != "Mul"
    for i in range(len(onnx_model.graph.node)):
                
    for i in range(len(onnx_model.graph.node)):
      
        if node_type in onnx_model.graph.node[i].name:
            new_name = f"{node_type}{i}"
            onnx_model.graph.node[i].name = new_name
      

In [ ]:
def only_Mul(name):
    return "Mul" in name and ("MatMul" not in name) and ("MultiThreshold" not in name)

In [ ]:
def rename_Mul(onnx_model):
    for i in range(len(onnx_model.graph.node)):
        
        for j in range(len(onnx_model.graph.node[i].input)):
            if only_Mul(onnx_model.graph.node[i].input[j]):
                index = find_i_by_prefix(onnx_model,onnx_model.graph.node[i].input[j])
                suffix = onnx_model.graph.node[i].input[j].split("_output_")[1]
                new_input_name = f"Mul{index}_output_{suffix}"
                onnx_model.graph.node[i].input[j] = new_input_name

        
        for j in range(len(onnx_model.graph.node[i].output)):
            if only_Mul(onnx_model.graph.node[i].output[j]):
                new_output_name = f"Mul{i}_output_{j}"
                onnx_model.graph.node[i].output[j] = new_output_name
                
    for i in range(len(onnx_model.graph.node)):
      
        if only_Mul(onnx_model.graph.node[i].name):
            new_name = f"Mul{i}"
            onnx_model.graph.node[i].name = new_name    

In [ ]:
rename_nodes_by_type(onnx_model, "MatMul")
rename_nodes_by_type(onnx_model, "MultiThreshold")
rename_nodes_by_type(onnx_model, "Add")
rename_nodes_by_type(onnx_model, "Softmax")
rename_nodes_by_type(onnx_model, "Transpose")
rename_nodes_by_type(onnx_model, "Unsqueeze")
rename_Mul(onnx_model)

In [ ]:
def constant_output_handler(onnx_model):
    for i in range(len(onnx_model.graph.node)):
        for j in range(len(onnx_model.graph.node[i].input)):
            if "Constant" in onnx_model.graph.node[i].input[j]:
                #print('-'*60)
                input_name = onnx_model.graph.node[i].input[j]
                #print(input_name)
                new_name = 
                
                for k in range(len(onnx_model.graph.initializer)):
                    init_name = onnx_model.graph.initializer[k].name
                    if input_name == init_name:
                        #print(init_name)
                        onnx_model.graph.initializer[i].name = f"{onnx_model.graph.node[i].name}_param0" 
                for l in range(len(onnx_model.graph.quantization_annotation)):
                    tensor_name = onnx_model.graph.initializer[k].tensor_name
                    if input_name == tensor_name:
                        #print(init_name)
                        tensor_name = onnx_model.graph.initializer[k].tensor_name = f"{onnx_model.graph.node[i].name}_param0"
                
                onnx_model.graph.node[i].input[j] = f"{onnx_model.graph.node[i].name}_param0" 


In [ ]:
constant_output_handler(onnx_model)

In [ ]:
onnx.save(onnx_model, 'oppo_attn_conv_3_mod.onnx')

In [ ]:
def rename_constant(model):
    graph = model.graph
    for n in graph.node:
        assert n.name != "", "Found empty node name"
        init_in_num = 0
        for i in n.input:
            if get_initializer(model, i) is not None:
                    rename_tensor(model, i, "%s_param%d" % (n.name, init_in_num))
                    init_in_num += 1

In [ ]:
rename_constant(onnx_model)